This notebook will be used to test the sensitivity of tidal ellipses from the model output based on data series variations such as length, frequency and seasonality.

In [2]:
import matplotlib.pyplot as plt
import numpy as np
import datetime
import netCDF4 as nc
from dateutil import tz
import os
from glob import glob

from salishsea_tools import(viz_tools, tidetools, nc_tools)
from salishsea_tools.nowcast import (analyze, figures, research_VENUS)
from IPython.display import Image
import datetime

%matplotlib inline

##1. Frequency
The model has been giving an ouput for tracers and velocities every 15 minutes since Myy 9th 2015. This section will compare the tidal ellipses of the hourly data and the quarter-hourly data between the 9th and 31st of May 2015.

The data has 